In [ ]:
# arrange videos side-by-side and add text
import os

videos = [
    "media/lowres_highsnr/FMLP/225/timecoded_cfr.mp4",
    "media/lowres_highsnr/FMLP/450/timecoded_cfr.mp4",
    "media/lowres_highsnr/FMLP/900/timecoded_cfr.mp4",
    "media/lowres_highsnr/BH/900/timecoded_cfr.mp4"
]
titles = ["T=225", "T=450", "T=900", "BH reference"]
w, h = 134, 134

N = len(videos)
command = "ffmpeg -y " + " ".join(["-i \"{}\"".format(v) for v in videos]) + " -filter_complex \"[0:v]pad=iw*{}:ih[int0]".format(N) # pad the first video
for i in range(1, N):
    command += ";[{}:v][0:v]scale2ref[scaled{}],nullsink;[int{}][scaled{}]overlay={}*W/{}:0[int{}]".format(i,i,i-1, i, i, N, i) # scale video to same dimensions as the first video and overlay it
command += ";[int{}]pad=iw:ih+30[text0]".format(N-1)

for i in range(N):
    command += ";[text{}]drawtext=fontfile=arial.ttf:text='{}':fontcolor=white:fontsize=15:x=({}-text_w/2):y=({}+15-text_h/2)[text{}]".format(i, titles[i], w/2 + w * i, h, i+1)
command += "\" -map '[text{}]' output.mp4".format(N)

print(command)

os.system(command)
#os.system("ffmpeg -y -i \"{}\" -i \"{}\" -i \"{}\" -i \"{}\" -filter_complex '[0:v]pad=iw*3:ih[int];[int][1:v]overlay=W/3:0[int2];[int2][2:v]overlay=2*W/3:0[vid]' -map '[vid]' output.mp4".format(videos[0], videos[1], videos[2], videos[3]))

In [ ]:
# convert to gif
import os
import glob

videos = glob.glob("media/**/timecoded_cfr.mp4", recursive=True)
for v in videos:
    output = os.path.join(os.path.dirname(v), "timecoded_cfr.gif")
    os.system("palette=\"/tmp/palette.png\"; filters=\"\"; ffmpeg -i {input} -vf \"palettegen\" -y $palette; ffmpeg -i \"{input}\" -i $palette -lavfi \"[0:v][1:v] paletteuse\" -y \"{out}\"; ".format(input=v, out=output))